In [6]:
import os,csv
import xml.etree.ElementTree as ET

In [25]:
# site = "stackoverflow"
# site = "arduino"
site = "raspberrypi"

In [3]:
only563 = []
file_name = site + "\only563.txt"
temp = []
with open(file_name) as f:
    temp = f.readlines()

only563 = []
for l in temp:
    only563.append(l.strip())
print(only563)

['16002', '21915', '52708', '5357', '56109', '59972', '61130', '61865', '63608', '65012', '6549', '66433', '69476', '71255', '72364', '72652', '75768', '76410', '78161', '85297']


load vulnerable post ids

In [4]:
vul_ids = []
vul_ids_file_name = site + "\VulIds.txt"
temp = []
with open(vul_ids_file_name) as f:
    temp = f.readlines()

vul_ids = []
for l in temp:
#     if l.strip() in only563:
#         continue
    vul_ids.append(l.strip())
    
print(len(vul_ids))

309


obtain list of posts (ids) with CHANGES (more than one entry in Version_Information.csv) and without changes

In [5]:
post_versions = {} #key: post_id, value: number of versions (1 = only original)
file_name = site +"/Version_Information.csv"
with open(file_name, mode='r', encoding="utf8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if row['PostId'] in vul_ids:
            if row['PostId'] in post_versions:
                post_versions[row['PostId']]+=1
            else:
                 post_versions[row['PostId']]=1

print(len(post_versions))

309


In [43]:
outputFile = open(site+"/postsWithRevisions.txt","a")
for k,v in post_versions.items():
    if v>1:
        outputFile.write(k + "\n")
#         print(k,v)
outputFile.close()   

In [44]:
outputFile = open(site+"/postsWithNoRevisions.txt","a")
for k,v in post_versions.items():
    if v==1:
        outputFile.write(k +"\n")
#         print(k,v)
outputFile.close()  

count number of SNIPPETS that have only one post version (i.e. part of ZERO code revisions count)

In [6]:
#load ids with no revisions
norev = []
temp = []
with open(site+"/postsWithNoRevisions.txt") as f:
    temp = f.readlines()

for l in temp:
    norev.append(l.strip())


count = 0
allsnips = set()
xml = ET.parse(site+"/output-new.xml")
root = xml.getroot()
errors = root[1]

for i in errors:
    key = i.get("id")
    if i.get("cwe") is not None:
        loc=i[0]
        fileName = loc.get("file")
        allsnips.add(fileName)
        
        
for i in allsnips:
    temp=i.split("-")
    postId = temp[1].split(".")
    line=postId[0]
    if line in norev:
        count+=1

print("Number of code snippets with no POST versions and therefore no CODE revisions: ",count)

#43+113=156 for SO

Number of code snippets with no POST versions and therefore no CODE revisions:  161


In [26]:
#count number of SNIPPETS that have >1 post version
rev = []
temp = []
with open(site+"/postsWithRevisions.txt") as f:
    temp = f.readlines()

for l in temp:
    rev.append(l.strip())


count = 0
allsnips = set()
xml = ET.parse(site+"/output-new.xml")
root = xml.getroot()
errors = root[1]

for i in errors:
    key = i.get("id")
#     print(key)
    if i.get("cwe") is not None and i.get("cwe") != "563":
        loc=i[0]
        fileName = loc.get("file")
        allsnips.add(fileName)
#         print(fileName)

for i in allsnips:
    temp=i.split("-")
    postId = temp[1].split(".")
    line=postId[0]
    if line in rev:
        temp = i.split(".")[0]
#         if "." in temp:
#             temp = temp.split(".")[0]
        print(temp[5:])
        count+=1

print(len(allsnips)) #104+ 274 = 378 total snippets

print("Number: ",count)
#63+161 = 224

113177
22057
26217-2
26746
59255
24890
24890-2
99180
130530-3
81747
43600
130530-2
24890-3
57446
124243-2
19767
12087
78348
7013
102431
40
Number:  20


determine revision type and CWE type correlation
determine revision counts and CWE type correlation

In [27]:
posts_cwes = {} #key=snippetid, value = cwes
posts_revisions = {} #key=snippetid, value = revision type
posts_revcounts = {} ##key=snippetid, value = revision count

file_name = site +"/revisiontypes.csv"
with open(file_name, mode='r', encoding="utf8") as f:
    reader = csv.DictReader(f)
    for row in reader:   
        if len(row['RevisionType'])!=0:
            posts_revisions[row['\ufeffSnippetId']] = row['RevisionType']
        
        if len(row['RevCount'])!=0:
            posts_revcounts[row['\ufeffSnippetId']] = row['RevCount']


# print(len(posts_revisions))

xml = ET.parse(site+"/output-new.xml")
root = xml.getroot()
errors = root[1]

for i in errors:
    loc=i[0]
    fileName = loc.get("file")
    sid = fileName[5:].split(".")[0] #snippet id
    if sid  in posts_revisions:
        if sid in posts_cwes:
            temp = posts_cwes[sid]
#             if i.get("cwe") not in temp:
            temp.append(i.get("cwe"))
            posts_cwes[sid] = temp
        else:
            temp = []
            temp.append(i.get("cwe"))
            posts_cwes[sid] = temp

print(posts_cwes)
            
# print(len(posts_cwes))

# for k,v in posts_cwes.items():
#     for i in v:
#         print(i,posts_revisions[k])

for k,v in posts_cwes.items():
    for i in v:
        print(i,posts_revcounts[k])

{'113177': ['686'], '19767': ['398'], '24890-3': ['398'], '57446': ['398'], '7013': ['398', '758', '190', '457', '457', '457'], '78348': ['398', '398'], '81747': ['398']}
686 1
398 1
398 1
398 2
398 1
758 1
190 1
457 1
457 1
457 1
398 1
398 1
398 1
